In [ ]:
# get the work directory
import os
os.getcwd()
# os.chdir('/')  change the work directory

In [ ]:
with open("C:\\Users\\User\\Desktop\\Hackathon\\camp_dataset\\sim_question_train.txt", encoding = "utf-8") as f:
    dat_train = f.readlines()

In [ ]:
with open("C:\\Users\\User\\Desktop\\Hackathon\\camp_dataset\\sim_question_test.txt", encoding = "utf-8") as f:
    dat_test = f.readlines()

In [ ]:
# import module
import numpy as np
import pandas as pd
import jieba
import gensim
from gensim import models

In [ ]:
## define the function that split the data
def split_train_data (text):
    text_pd = pd.DataFrame(text)
    text_pd.columns = ["question"]
    question = text_pd["question"].str.split("@@@@@", expand = True)
    text_pd["q1"] = question[0]
    text_pd["q2"] = question[1]
    text_pd["pair"] = question[2]
    text_pd.drop("question", axis = 1, inplace =True)
    return text_pd

def split_test_data (text):
    text_pd = pd.DataFrame(text)
    text_pd.columns = ["question"]
    question = text_pd["question"].str.split("@@@@@", expand = True)
    text_pd["q1"] = question[0]
    text_pd["q2"] = question[1]
    text_pd.drop("question", axis = 1, inplace =True)
    return text_pd

In [ ]:
data_train = split_train_data(dat_train)
data_test = split_test_data(dat_test)

In [ ]:
# first remove the \n in the train pair column
data_train["pair"] = data_train["pair"].replace(r"\n", "",regex = True)
data_train["pair"] = data_train["pair"].str.strip()

In [ ]:
# define the function to clean data
def clean_data (data):
    # remove the meaningless signs
    for column in data:
        # remove $
        data[column] = data[column].str.replace("$", "")  
        
        #remove the numbers, the alphabets
        #data[column] = data[column].str.replace("\d+", "").str.replace(r'[A-Za-z]+', '') 
        
        # remove the calculation sign (like +, -, *, (, { )
        data[column] = data[column].apply(lambda x: x.replace("+", "").replace("-", "").replace("=", "").replace("*", ""))
        data[column] = data[column].apply(lambda x: x.replace("{","").replace("}","").replace("、","").replace("：","").replace("？","").replace("（","").replace("）","").replace("，","").replace(".",""))
        data[column] = data[column].apply(lambda x: x.replace("(","").replace(")","").replace("?","").replace("。","").replace("|","").replace(",","").replace("^","").replace("；",""))
        
        # remove the white space and \xa0 within sentences
        data[column] = data[column].str.replace(" ", "").str.replace(u'\xa0', "")
        
        # remove white spaces
        data[column] = data[column].str.strip() 
        
    return data

In [ ]:
# clean dataset
data_train = clean_data(data_train)
data_test = clean_data(data_test)
#data_train.head(500)

In [ ]:
## define tokenize function
def tokenize_question (data, column):
    q_words = []
    for i in data[column]:
        segs = jieba.cut(i)
        result = " ".join(segs).split(" ")
        q_words.append(result)
    return q_words

In [ ]:
# tokenize the train and test data
q1_words = tokenize_question(data_train, "q1")
q2_words = tokenize_question(data_train,"q2")
q1_test_words = tokenize_question(data_test, "q1")
q2_test_words = tokenize_question(data_test, "q2")

In [ ]:
words_combine = []
vic = dict()
for i in q1_words:
    words_combine.append(i)
for i in q2_words:
    words_combine.append(i)
for i in q1_test_words:
    words_combine.append(i)
for i in q2_test_words:
    words_combine.append(i)

In [ ]:
## use model to train word embedding
#model1 = gensim.models.Word2Vec(words_combine, size=100, window=5, min_count=1)    ## accuracy = 0.690
#model2 = gensim.models.Word2Vec(words_combine, size=200, window=5, min_count=1)     ## accuracy = 0.688
#model3 = gensim.models.Word2Vec(words_combine, size=150, window=5, min_count=1)      ## accuracy = 0.6878
#model4 = gensim.models.Word2Vec(words_combine, size=100, window=8, min_count=1)    ## accuracy = 0.708
model5 = gensim.models.Word2Vec(words_combine, size=100, window=10, min_count=1)    ## accuracy = 0.711
#model6 = gensim.models.Word2Vec(words_combine, size=100, window=12, min_count=1)    ## accuracy = 0.716

In [ ]:
## use model on each pair question
pred_dic = {"q1":q1_words, "q2":q2_words}
pred_df = pd.DataFrame(pred_dic)
def compare_similarity(x, y):
    return model1.n_similarity(x,y).item()

pred_df['similarity'] = pred_df.apply(lambda row: compare_similarity(row['q1'], row['q2']), axis=1)

In [ ]:
## evaluate the accuracy
pred_df["pred_pair"] = pred_df["similarity"].apply(lambda x: 1 if x >=0.5 else 0)
pred_df["true_pair"] = data_train["pair"].astype(str).astype(int)
result = pred_df[pred_df["true_pair"] == pred_df["pred_pair"]]
accuracy = len(result)/len(pred_df)
print(accuracy)

In [ ]:
## predict test set
test_data_dic = {"q1":q1_test_words, "q2":q2_test_words}
test_data_df = pd.DataFrame(test_data_dic)
def compare_similarity(x, y):
    return model5.n_similarity(x,y).item()

test_data_df['similarity'] = test_data_df.apply(lambda row: compare_similarity(row['q1'], row['q2']), axis=1)
test_data_df["pred_pair"] = test_data_df["similarity"].apply(lambda x: 1 if x >=0.5 else 0)